In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io, requests
import openpyxl

In [2]:
NammaMetro_Stations = pd.read_csv("NammaMetro_Stations_Dataset.csv")
NammaMetro_Stations = NammaMetro_Stations[NammaMetro_Stations['code'] != '[TBC]'].reset_index(drop=True)
display(NammaMetro_Stations)

,code,name_eng,name_kan,lines,is_terminus,is_interchange
0,AGPP,Attiguppe,ಅತ್ತಿಗುಪ್ಪೆ,🟣,False,False
1,BTML,BTM Layout,ಬಿ ಟಿ ಎಂ ಬಡಾವಣೆ,🟡,False,False
2,BYPL,Baiyappanahalli,ಬೈಯ್ಯಪ್ಪನಹಳ್ಳಿ,🟣,False,False
3,BSNK,Banashankari,ಬನಶಂಕರಿ,🟢,False,False
4,JTPM,Benniganahalli,ಬೆನ್ನಿಗಾನಹಳ್ಳಿ,🟣,False,False
...,...,...,...,...,...,...
78,VJRH,Vajarahalli,ವಾಜರಹಳ್ಳಿ,🟢,False,False
79,VJN,Vijayanagar,ವಿಜಯನಗರ,🟣,False,False
80,WHTM,Whitefield (Kadugodi),ವೈಟ್ ಫೀಲ್ಡ್ (ಕಾಡುಗೋಡಿ),🟣,True,False
81,PUTH,Yelachenahalli,ಯಲಚೇನಹಳ್ಳಿ,🟢,False,False


In [3]:
df_hourly = pd.read_excel("https://github.com/Vonter/bmrcl-ridership-hourly/raw/8c569cc40f97ac9f5327b1b32432af2cb6cbacc2/raw/station-hourly.xlsx")

station_name_fixes = {
    'SV Road': 'Swami Vivekananda Road',
    'MG Road': 'Mahatma Gandhi Road',
    'Vidhana Soudha': 'Dr. B. R. Ambedkar Station, Vidhana Soudha',
    'Sir.M. Visveshwaraya': 'Sir.M.Visvesvaraya Stn., Central College',
    'Kempegowda': 'Nadaprabhu Kempegowda Station, Majestic',
    'Bangalore City Station': 'Krantivira Sangolli Rayanna Railway Station',
    'Hosahalli': 'Sri Balagangadharanatha Swamiji Station, Hosahalli',
    'Nayandahalli': 'Pantharapalya–Nayandahalli',
    'Pattangere': 'Pattanagere',
    'Krishnarajapura': 'Krishnarajapura (K.R.Pura)',
    'Seetharampalya': 'Seetharamapalya',
    'Nallur Halli': 'Nallurhalli',
    'Whitefield (Kadugodi)': 'Whitefield (Kadugodi)',
    'Yeshwanthpur Industry': 'Goraguntepalya',
    'Kuvempu Road': 'Mahakavi Kuvempu Road',
    'KR Market': 'Krishna Rajendra Market',
    'RV Road': 'Rashtreeya Vidyalaya Road',
    'Banashankarl': 'Banashankari',
    'JP Nagar': 'Jaya Prakash Nagar',
    'Puttenahalli': 'Yelachenahalli',
    'Thalghattapura': 'Thalaghattapura',
    'Manjunatha Nagar': 'Manjunathanagara',
}

for col in df_hourly.columns.tolist()[2:-1]:
    col_reformatted = col.split()[0][:2]
    df_hourly[col_reformatted] = df_hourly[col]

df_hourly['date'] = pd.to_datetime(df_hourly['BUSINESS DATE'], format='%Y-%m-%d')
df_hourly['station_name'] = df_hourly['STATION'].str.split('-').str[1]
df_hourly['station_code'] = df_hourly['STATION'].str.split('-').str[0]

df_hourly = df_hourly[['date', 'station_code', 'station_name'] + [f"{n:02d}" for n in range(24)] + ['TOTAL']].copy()
df_hourly['total_check'] = df_hourly['TOTAL'] == df_hourly[[f"{n:02d}" for n in range(24)]].sum(axis=1)
df_hourly = df_hourly.loc[:, :'TOTAL'] if df_hourly.loc[:, 'total_check'].isna().sum() == 0 else df_hourly

df_hourly['station_name'] = df_hourly['station_name'].replace(station_name_fixes)

condition = dict(zip(NammaMetro_Stations['name_eng'], NammaMetro_Stations['code']))
df_hourly['station_code'] = df_hourly['station_name'].map(condition)

display(df_hourly)

,date,station_code,station_name,00,01,02,03,04,05,06,...,15,16,17,18,19,20,21,22,23,TOTAL
0,2025-08-01,BYPL,Baiyappanahalli,0,0,0,0,19,83,215,...,456,497,880,942,923,489,373,134,15,13836
1,2025-08-01,SVRD,Swami Vivekananda Road,0,0,0,0,7,39,85,...,573,866,1078,1068,856,433,211,132,26,9067
2,2025-08-01,IDN,Indiranagar,0,0,0,0,4,41,175,...,1301,1660,2303,3456,2982,1866,1248,712,161,24471
3,2025-08-01,HLRU,Halasuru,0,0,0,0,5,51,192,...,418,387,566,756,660,580,317,151,17,9924
4,2025-08-01,TTY,Trinity,0,0,0,0,2,17,50,...,537,838,1899,3404,2572,1093,421,319,38,13746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,2025-08-18,ETCT,Electronic City,0,0,0,0,0,15,29,...,207,242,235,261,178,113,95,43,0,2761
1351,2025-08-18,ECTN,Infosys Foundation Konappana Agrahara,0,0,0,0,10,35,129,...,346,474,594,574,411,282,165,45,0,5534
1352,2025-08-18,HSKR,Huskur Road,0,0,0,0,0,6,53,...,107,171,147,207,128,77,51,37,0,1954
1353,2025-08-18,HBGI,Biocon Hebbagodi,0,0,0,0,0,5,37,...,73,93,113,138,80,59,25,15,0,1461


In [4]:
df_pairhourly = pd.read_excel('https://github.com/Vonter/bmrcl-ridership-hourly/raw/8c569cc40f97ac9f5327b1b32432af2cb6cbacc2/raw/stationpair-hourly.xlsx')

station_code_fixes = {
    'BNSK': 'BSNK',
    'BRCS': 'SRCS',
    'BYPH': 'BYPL',
    'CHPK': 'CKPE',
    'CLG': 'CLGA',
    'CLGT': 'CLGA',
    'JAYN': 'JYN',
    'JHLI': 'JLHL',
    'JIDL': 'CKBD',
    'KRMA': 'KRAM',
    'KRPM': 'KRAM',
    'LAGH': 'LBGH',
    'MGRD': 'MAGR',
    'PATC': 'PATG',
    'RVRD': 'RVR',
    'VJNR': 'VJN',
}

df_pairhourly['STATION'] = df_pairhourly['STATION'].map(station_code_fixes).fillna(df_pairhourly['STATION'])

df_pairhourly = df_pairhourly.rename(columns={"BUSINESS DATE": "date", "STATION": "station_code"})
df_pairhourly = df_pairhourly.rename(columns=station_code_fixes)

df_pairhourly['hour'] = df_pairhourly['date'].str.split('-').str[3].str[:2].astype(int)
df_pairhourly['date'] = df_pairhourly['date'].str.split('-').str[0:3].str.join('-')
df_pairhourly['date'] = pd.to_datetime(df_pairhourly['date'], format='%Y-%m-%d')

df_pairhourly['lines'] = df_pairhourly['station_code'].map(NammaMetro_Stations[NammaMetro_Stations['code']!='[TBC]'].set_index('code')['lines']).copy()

df_pairhourly.dropna(subset=['station_code'], inplace=True)
df_pairhourly = df_pairhourly[['date', 'hour', 'station_code', 'lines']+df_pairhourly.columns[2:-2].tolist()]

display(df_pairhourly)

,date,hour,station_code,lines,BYPL,SVRD,IDN,HLRU,TTY,MAGR,...,OFDC,MSRN,CKBR,BSRD,HOSR,ETCT,ECTN,HSKR,HBGI,BMSD
0,2025-08-01,4,MIRD,🟣,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2025-08-01,5,BYPL,🟣,0,2,8,1,3,14,...,0,0,0,0,0,0,0,0,0,0
4,2025-08-01,5,SVRD,🟣,0,0,3,3,1,0,...,0,0,0,0,0,0,0,0,0,0
5,2025-08-01,5,IDN,🟣,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,2025-08-01,5,HLRU,🟣,1,0,1,0,3,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27509,2025-08-18,23,ETCT,🟡,0,0,0,0,0,2,...,0,0,0,0,0,0,2,4,0,0
27510,2025-08-18,23,ECTN,🟡,0,0,2,0,0,2,...,0,2,0,0,0,0,0,0,0,0
27511,2025-08-18,23,HSKR,🟡,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
27512,2025-08-18,23,HBGI,🟡,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
inspection_date = '2025-08-18'
inspection_hour = -1

pairhourly_filtered = lambda inspection_date, inspection_hour: \
    df_pairhourly[(df_pairhourly['date'] == inspection_date) 
                & (df_pairhourly['hour'] == inspection_hour if inspection_hour >= 0 else True)].copy()
df_pairhourly_filtered = pairhourly_filtered(inspection_date, inspection_hour)

# with pd.option_context("display.max_rows", None, "display.max_columns", None): 
display(df_pairhourly_filtered)

In [ ]:
def inspect_same_in_and_out(df):
    same_in_and_out_df = NammaMetro_Stations.copy()
    same_in_and_out_df['sio_count'] = [0] * len(same_in_and_out_df)
    same_in_and_out_df = same_in_and_out_df.reset_index(drop=True)

    for idx in df.index:
        station = df.loc[idx, 'station_code']
        count = int(df.loc[idx, station])
        same_in_and_out_df.loc[same_in_and_out_df['code']==station, 'sio_count'] += count

    same_in_and_out_df = same_in_and_out_df.set_index('code').T
    same_in_and_out_df = same_in_and_out_df[df.columns.tolist()[4:]]
    same_in_and_out_df = same_in_and_out_df.T.reset_index()

    return same_in_and_out_df

same_in_and_out_df = inspect_same_in_and_out(df_pairhourly_filtered)
with pd.option_context("display.max_rows", None, "display.max_columns", None): 
    display(same_in_and_out_df)

In [ ]:
print("Same Station In & Out")
print("🟣", same_in_and_out_df[same_in_and_out_df['lines'].str.contains('🟣')]['sio_count'].sum())
print("🟢", same_in_and_out_df[same_in_and_out_df['lines'].str.contains('🟢')]['sio_count'].sum())
print("🟡", same_in_and_out_df[same_in_and_out_df['lines'].str.contains('🟡')]['sio_count'].sum())

In [ ]:
print(f"Total sio_count = {same_in_and_out_df['sio_count'].sum()}")

same_in_and_out_df_sorted = same_in_and_out_df.sort_values(by='sio_count', ascending=False)
same_in_and_out_df_sorted = same_in_and_out_df_sorted[same_in_and_out_df_sorted['sio_count'] > 0]
same_in_and_out_df_sorted = same_in_and_out_df_sorted.reset_index(drop=True)

display(same_in_and_out_df_sorted)

In [ ]:
date_sequence = df_pairhourly['date'].dt.strftime('%Y-%m-%d').unique()
n_rows, n_cols = len(date_sequence), len(NammaMetro_Stations)

selfpairs = []
for inspection_date in date_sequence:
    selfpairs.append(inspect_same_in_and_out(pairhourly_filtered(inspection_date, -1))['sio_count'].values)

selfpairs = np.array(selfpairs)

print(f"selfpairs matrix: {selfpairs.shape[0]} dates x {selfpairs.shape[1]} stations")

In [ ]:
selfpairs_rowsum = np.sum(selfpairs, axis=1)
selfpairs_rowsum_df = pd.DataFrame({"date": date_sequence, "sio_total": selfpairs_rowsum})
selfpairs_rowsum_df

In [ ]:
selfpairs_colsum = np.sum(selfpairs, axis=0)
selfpairs_colsum_df = pd.DataFrame({"station": df_pairhourly.columns[4:], "sio_total": selfpairs_colsum})
selfpairs_colsum_df.sort_values(by="sio_total", ascending=False).reset_index(drop=True)

In [ ]:
print(f"{np.sum(selfpairs):3,} riders entered and exited through\n\
the same station with a single ticket\n\
between {date_sequence[0]} and {date_sequence[-1]}.")

In [ ]:
dates = df_pairhourly['date'].dt.strftime('%Y-%m-%d').unique().tolist()
active_stations = df_pairhourly.columns[4:].tolist()
station_to_idx = {station: idx for idx, station in enumerate(active_stations)}

data = np.zeros((len(dates),    # days of observations 
                         24,    # hours in a day
       len(active_stations),    # max stations by row
       len(active_stations)))   # max stations by column

for i, date in enumerate(dates):
    date_mask = df_pairhourly['date'] == date
    for hour in range(24):
        hour_mask = df_pairhourly['hour'] == hour
        subset = df_pairhourly[date_mask & hour_mask]
        
        if len(subset) > 0:
            for _, row in subset.iterrows():
                origin_idx = station_to_idx[row['station_code']]
                # Fill in the values for this origin station to all destinations
                data[i, hour, origin_idx, :] = row[active_stations].values

In [ ]:
# Total riders per day (all rides)
daily_totals = data.sum(axis=(1, 2, 3))  # Sum over hours, origins, destinations
daily_totals

In [18]:
# Total riders per hour across all days (hourly pattern)
hourly_totals = data.sum(axis=(0, 2, 3))  # Sum over days, origins, destinations
hourly_totals

array([1.075100e+04, 1.000000e+00, 1.000000e+00, 0.000000e+00,
       1.356000e+03, 4.021000e+04, 1.114030e+05, 2.841400e+05,
       6.643330e+05, 1.037235e+06, 9.560080e+05, 6.799650e+05,
       6.365220e+05, 6.428180e+05, 6.459770e+05, 6.297040e+05,
       7.338820e+05, 9.169940e+05, 1.110150e+06, 1.115761e+06,
       8.107790e+05, 5.511350e+05, 3.419090e+05, 1.384410e+05])

In [15]:
# Total departures from each station (all days/hours)
origin_totals = data.sum(axis=(0, 1, 3))  # Sum over days, hours, destinations
origin_totals

array([231540., 150872., 380598., 157695., 198268., 372369., 234919.,
       202608., 201014., 626056., 218713., 128828., 165216., 197150.,
       143551.,  76622., 105087.,  39530., 101356.,  72947.,  96866.,
       127011., 103397., 113899., 469307., 327496., 146984., 128414.,
       124532., 149585., 136644., 137180., 137698., 179322., 207345.,
        45401., 150949., 193719., 201929., 145732.,  38492.,  68484.,
       134283., 251692., 215184., 202223., 188091.,  94201., 101847.,
       295139., 284844., 167724., 203522., 149814., 139068., 237281.,
       155060., 227408., 177215., 165904., 224469.,  89553.,  91161.,
        50102., 123020.,  22514.,  71558., 132172.,  22879.,  35917.,
        28968.,  38572.,  34656.,  22491.,  29101.,  11560.,  29802.,
         5303.,  28272.,  50774.,  18705.,  14195.,  57906.])

In [14]:
# Total arrivals at each station (all days/hours)
destination_totals = data.sum(axis=(0, 1, 2))  # Sum over days, hours, origins
destination_totals

array([214385., 152995., 401228., 153559., 194771., 383640., 236131.,
       203390., 213070., 842201., 248569., 128759., 141882., 217962.,
       137499.,  94975.,  93710.,  38036., 108548.,  71914.,  92222.,
       134684.,  53137.,  95314., 412103., 304195., 148823., 137710.,
       117272., 142208., 138446., 135397., 133049., 177797., 190818.,
        46055., 146708., 160413., 187679., 142878.,  35634.,  60124.,
       115870., 258883., 194575., 196547., 201516.,  92218., 100608.,
       297666., 305663., 167697., 213447., 144852., 131896., 244231.,
       157739., 234163., 178964., 162300., 221443.,  87914.,  92712.,
        49036., 104130.,  20748.,  67463., 119669.,  22644.,  33377.,
        36508.,  40298.,  35725.,  20812.,  27592.,  11663.,  30288.,
         5068.,  31886.,  43600.,  16116.,  11949.,  60109.])

In [13]:
# Origin-destination matrix (all days/hours combined)
od_matrix = data.sum(axis=(0, 1))  # Shape: (83, 83)

In [10]:
# Extract diagonal (same origin and destination) for each day-hour
same_station_rides = np.diagonal(data, axis1=2, axis2=3)
# Shape: (18 days, 24 hours, 83 stations)

# Sum across hours and stations to get daily totals
day_totals_same_station = same_station_rides.sum(axis=(1, 2))
# Shape: (18,) - one value per day

# Print daily totals
for day_idx, total in enumerate(day_totals_same_station):
    print(f"{dates[day_idx]}: {int(total):,}")

2025-08-01: 1,999
2025-08-02: 2,248
2025-08-03: 2,033
2025-08-04: 2,192
2025-08-05: 1,912
2025-08-06: 2,004
2025-08-07: 1,994
2025-08-08: 1,876
2025-08-09: 2,303
2025-08-10: 2,367
2025-08-11: 3,312
2025-08-12: 2,613
2025-08-13: 2,910
2025-08-14: 3,684
2025-08-15: 5,847
2025-08-16: 3,117
2025-08-17: 2,933
2025-08-18: 2,936
